In [0]:
#%pip install -r requirements.txt

In [0]:
#%pip install pandas fsspec adlfs

In [0]:
#dbutils.library.restartPython()

In [0]:
#%run ./_paramgplayscrapper

In [0]:
#%run ./_envsettings

In [0]:
#Libraries
import pandas as pd
import numpy as np 
import io
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [0]:
df = pd.read_csv(
    f"abfss://root@{account_name}.dfs.core.windows.net/{sentiment_analyzer_input_file}",
    storage_options={
        "account_name": account_name,
        "account_key": account_key
    }
)

In [0]:
#df=df.toPandas()
# df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['at'] = pd.to_datetime(df['at'], errors='coerce')

# df=df[['review','date','rating']]
# df.rename(columns={'review': 'Review', 'date': 'Date','rating':'Rating'}, inplace=True)
df = df[['content', 'at', 'score']]
df.rename(columns={'content': 'Review', 'at': 'Date', 'score': 'Rating'}, inplace=True)

nltk.download('vader_lexicon')

sentiment_analyzer = SentimentIntensityAnalyzer()
 
df[document_id_column] = range(1, len(df) + 1)

In [0]:
def calculate_sentiment(review):
    if review is None:
        # Handling None values by returning a neutral compound score
        return 0.0
    scores = sentiment_analyzer.polarity_scores(review)
    compound_score = scores['compound']
    return compound_score

In [0]:
df['Scores'] = df['Review'].apply(calculate_sentiment)

In [0]:
def classify_sentiment_threshold(score):
    if score >= positive_threshold:
        return 'Positive'
    elif score <= negative_threshold:
        return 'Negative'
    else:
        return 'Neutral'

In [0]:
df['Sentiment'] = df['Scores'].apply(classify_sentiment_threshold)
b=df.columns.to_list()
mod = [i.rstrip() for i in b]
mod = [i.replace(' ','_') for i in mod ]
mod_dict= dict(zip(b,mod))
df.rename(columns= mod_dict,inplace=True)
df = df.dropna(subset=['Date','Review'])
df = df.drop_duplicates(subset=['Review'], keep='first')
df = df[~df['Review'].str.lower().isin(['good','Good'])]

In [0]:
df.to_csv(
    f"abfss://root@{account_name}.dfs.core.windows.net/{sentiment_analyzer_output_file}",
    index=False,
    storage_options={"account_name": account_name, "account_key": account_key},
)